LIBRARY

In [ ]:
import requests
import pandas as pd
import requests
import time
import urllib
import glob
from bs4 import BeautifulSoup
import json
import csv
from pandasql import sqldf
from sqlalchemy import create_engine
from functools import *
import mysql.connector as msql
from mysql.connector import Error
import numpy as np
import pymysql
import time
import threading
import queue

***INTERNAL FUNCTION***

In [ ]:
 ### Find value of key in nested json 
def find_value_of_key(value, key):
    val = None
    if isinstance(value,dict):
        if key in value:
            return value[key]
        for i in value.items():
            val = find_value_of_key(i[1], key)
            if val is not None:
                return val
    if isinstance(value,list):
        for i in value:
            val = find_value_of_key(i, key)
    return val

**CRAWLING CATEGORIES FROM ONE PAGE**

In [ ]:
def get_product_catagories(url):
    http_headers = {
        'User-Agent':  'PostmanRUntime/7.29.2',
    }
    response = requests.get(url=url, headers=http_headers)
    parser = BeautifulSoup(response.text, 'html.parser', )
    product_box = parser.findAll('script')
    product_catagories = []
    for product in product_box:
        try:
            item = json.loads(str(product.contents[0]))
        except:
            item = {}
        product_val = find_value_of_key(item,"breadcrumbs")
        if product_val is not None:
            [product_catagories.append(i) for i in product_val if i["category_id"] != 0] 
    return product_catagories


### Testing the Functional
get_product_catagories('https://tiki.vn/dien-thoai-samsung-galaxy-z-flip-4-8gb-128gb-hang-chinh-hang-p194034217.html?itm_campaign=CTP_YPD_TKA_PLA_UNK_ALL_UNK_UNK_UNK_UNK_X.163088_Y.1744616_Z.3253348_CN.Product-Ads-B4Q4&itm_medium=CPC&itm_source=tiki-ads&spid=194034225')


#GET ID OF PRODUCT

In [ ]:
def get_max_page_number(name:str, categories_id:int):
    http_headers = {
    'User-Agent':  'PostmanRUntime/7.29.2',
    }
    url = f'https://tiki.vn/{name}/c{categories_id}'
    def find_max_tag(a,b):
        if isinstance(a,int):
            a_num = a
        else:
            a_tag = a.find('a')
            a_num = a_tag.get('data-view-label') if a_tag.get('data-view-label') is not None else 0
        b = b.find('a')
        b_num = b.get('data-view-label') if b.get('data-view-label') is not None else 0
        return max(int(a_num), int(b_num))
    
    div_tag = None
    while(div_tag is None):
        response = requests.get(url=url, headers=http_headers)
        parser = BeautifulSoup(response.text, 'html.parser', )
        div_tag = parser.find("div", {"data-view-id":"product_list_pagination_container"})   
    # find all 
    ul_tag = div_tag.find('ul')
    li_tag = ul_tag.findAll('li')
    return reduce(find_max_tag,li_tag)

### Testing the Functional
get_max_page_number("Điện thoại Smartphone", 1795)

*** CRAWLING ALL ID FROM CATEGORIES ***

In [ ]:
def crawling_product_id(name:str, categories_id:int, page_number:int):
    http_headers = {
        'User-Agent':  'PostmanRUntime/7.29.2',
    }
    product_list = set()
    for i in range(page_number):
        url =f'https://tiki.vn/{name}/c{categories_id}?page{i}'
        response = None
        while(response is None):
            response = requests.get(url=url, headers=http_headers)
        parser = BeautifulSoup(response.text, 'html.parser')
    
        # find all 
        product_box = parser.findAll('a')
        product_box = [[]] + list(product_box)
        res = set(reduce(get_product_id,product_box))
        product_list.update(res)
    return list(product_list)

 
def get_product_id(pre_val,product):
    data = product.get("data-view-content")
    if data is not None:
        item = json.loads(data)
        id = item['click_data'].get('id')
        if id is not None:
            pre_val.append(id)
    return pre_val

### Testing the Functional
crawling_product_id("Điện thoại Smartphone", 1795,12)

**CALL API TO GET PRODUCT INFORMATION, CATEGORIES**

In [ ]:
def get_data(product_id:int):
    http_headers = {
    'User-Agent':  'PostmanRUntime/7.29.2',
    }
    url = f'https://tiki.vn/api/v2/products/{product_id}'
    response = requests.get(url, headers = http_headers)
    result = response.json()
    categories = result.get('breadcrumbs')
    return response.json(), categories
    
data, categories = get_data(177746939)


**DUMP DATA TO FILE AND EXTRACT CATEGORIES**

In [ ]:
### DATA FEILD 
f_brand = ['id','name','slug'] ##### brand
f_brand_crawl = [{'brand':f_brand}]
f_seller = ['id','name','store_id','is_best_store'] ##### current_seller, other_seller
f_seller_crawl = [{'current_seller': f_seller},{'other_seller': f_seller}]
f_product = ['id','name', 'short_url', 'short_description', 'price', 'list_price', 'original_price', 'discount', 'discount_rate',
            'rating_average', 'inventory_status', 'inventory_type', 'is_fresh', 'seller','is_gift_card','data_version',
            'brand', 'current_seller','other_sellers','inventory','breadcrumbs','amplitude']
f_categories = ['url', 'name', 'category_id']

In [ ]:
#### DUMP HEADER TO FILE
def convert_product_feild(feild):
    res = []
    for x in feild:
        if isinstance(x,dict):
            for k,v in x.items():
                mini_res = [k +'_'+ i for i in v]
            res = res  + mini_res
        else:
            res.append(x)
    return res

product_header = convert_product_feild(f_product)
with open('Data/product.csv', 'w') as f:
    writing_data= csv.writer(f, delimiter=',')
    writing_data.writerow(product_header)

with open('Data/categories.csv', 'w') as f:
    writing_data= csv.writer(f, delimiter=',')
    writing_data.writerow(f_categories)

with open('Data/sellers.csv', 'w') as f:
    writing_data= csv.writer(f, delimiter=',')
    writing_data.writerow(f_seller)

In [ ]:
#### GET DATA AND DUMP TO FEILD
def extract_data_to_feild(data, feild, res = {}):
    for i in feild:
            res[i] = extract_data(data,i)
    return res
    
def extract_data(data,key):
    value = data.get(key)
    if value is None:
        value = find_value_of_key(data,key)
    return value
####TESTING FUNCTIONS
data, categories = get_data(192455760)
extract_data_to_feild(data, f_product)

In [ ]:
#### pandas extract data
def extract_product_data(data):
    c_seller_id = 0
    if data['current_seller'] is not None:
        c_seller_id = data['current_seller']['id']
    o_seller_id = None
    if data['other_sellers'] is not None:
        o_seller_id = [item['id'] for item in data['other_sellers']]
    brand_id = data['brand']['id']
    inventory_st  = data['inventory']
    amplitude = data['amplitude']

    del data['amplitude']
    del data['inventory']

    data['other_sellers'] = o_seller_id
    data['current_seller'] = c_seller_id
    data['brand'] = brand_id
    data["is_fresh"] = 1 if data["is_fresh"] == True else 0
    data["is_gift_card"] = 1 if data["is_gift_card"] == True else 0

    data['categories'] = data['breadcrumbs']
    del data['breadcrumbs']
    if amplitude is not None:
        data.update(amplitude)
    if inventory_st is not None:
        data.update(inventory_st)
    
    df = pd.json_normalize(data)
    return df

data, categories = get_data(123345348)
f_data = extract_data_to_feild(data, f_product)
df = extract_product_data(f_data)
pd.DataFrame(df)

In [ ]:
data, categories = get_data(192455760)
f_data = extract_data_to_feild(data, f_product)
df = extract_product_data(f_data)
list(df.columns.values)

In [ ]:
####DUMP DATA TO FILE
def dump_to_file(data,filename:str):
    with open(f'Data/{filename}.csv', 'a') as f:
        writing_data= csv.writer(f, delimiter=',')
        writing_data.writerow(data)

#### TESTING FUNC

***DUMP DATA TO MYSQL***

In [ ]:
###create engine
username = 'minhdung'
password = '020920'
host = 'localhost'

DB_NAME = 'Tiki_Product'

engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}")

with engine.connect() as conn:
    try:
        # Do not substitute user-supplied database names here.
        conn.execute(f"CREATE DATABASE IF NOT EXISTS {DB_NAME}")
    except Error as e:
        print("Error while connecting to MySQL", e)

***CREATE TABLE IF NOT EXISTS***

In [ ]:
def create_table(database:str, table:str, datatype:str):
    try:
        username = 'minhdung'
        password = '020920'
        host = 'localhost'
        engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}/{database}")
        with engine.connect() as conn:
            conn.execute(f'DROP TABLE IF EXISTS {table};')
            conn.execute(f'CREATE TABLE {table}({datatype})')
            print('Created table....')
    except Error as e:
            print("Error while connecting to MySQL", e)

***create categories table***

In [ ]:
table = "categories"
datatype = '''url varchar(255),
            name varchar(255),
            category_id INT(255),
            PRIMARY KEY (category_id)'''
create_table("Tiki_Product",table=table,datatype=datatype)
print("Table is created....")

***create seller table***

In [ ]:
table = "seller"
datatype = '''id INT(255),
            name varchar(255),
            store_id INT(255),
            is_best_store BOOLEAN,
            PRIMARY KEY (id)'''
create_table("Tiki_Product",table=table,datatype=datatype)
print("Table is created....")

***create product table***

In [ ]:
table = "product"
datatype = '''id INT(255),
            name VARCHAR(255),
            short_url VARCHAR(255),
            short_description VARCHAR(255),
            price BIGINT(255),
            list_price BIGINT(255),
            original_price BIGINT(255),
            discount INT(255),
            discount_rate FLOAT(255, 2),
            rating_average FLOAT(255,2),
            inventory_status VARCHAR(255),
            inventory_type VARCHAR(255),
            is_fresh tinyint(1),
            seller VARCHAR(255) ,
            is_gift_card tinyint(1),
            data_version INT(255),
            brand INT(255),
            current_seller INT(225) default NULL,
            other_sellers VARCHAR(255),
            categories LONGTEXT,
            has_freeship_plus_benefit VARCHAR(255),
            partner_rewards_amount FLOAT(255,2),
            tiki_rewards_amount FLOAT(255,2),
            total_rewards_amount FLOAT(255,2),
            product_virtual_type VARCHAR(255),
            fulfillment_type VARCHAR(255),
            PRIMARY KEY (id)'''
create_table("Tiki_Product","product",datatype)
print("Table is created....")
username = 'minhdung'
password = '020920'
host = 'localhost'
engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}/Tiki_Product")
with engine.connect() as conn:
    A =conn.execute(f'SELECT DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA = \'Tiki_Product\' AND table_name = \'{table}\' AND COLUMN_NAME = \'is_fresh\' ;')
    print(A.fetchall())

In [ ]:
def read_csv_to_dataframe(filename):
    df = pd.read_csv(f'Data/{filename}.csv', index_col=False, delimiter = ',')
    df = df.replace({np.NaN:None})
    return df
df = read_csv_to_dataframe('product')
df

***DUMP DATA TO SQL***

In [ ]:
def import_data_mysql(Dataframe:pd.DataFrame, database):
    try:
        username = 'minhdung'
        password = '020920'
        host = 'localhost'
        engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}/{database}")
        Dataframe["is_fresh"] = Dataframe["is_fresh"].astype(int)
        Dataframe["is_gift_card"] = Dataframe["is_gift_card"].astype(int)
        Dataframe.to_sql('temporary_table', con = engine.connect(), if_exists= 'replace',index = False)
        with engine.begin() as cnx:
            insert_sql = 'INSERT IGNORE INTO product (SELECT * FROM temporary_table)'
            cnx.execute(insert_sql)                  
    except Error as e:
            print("Error while import data to MySQL", e)


In [ ]:
def import_categories(Dataframe:pd.DataFrame, database):
    try:
        username = 'minhdung'
        password = '020920'
        host = 'localhost'
        Dataframe.drop(columns = 'status')
        engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}/{database}")
        with engine.begin() as cnx:
            Dataframe.to_sql('categories', con = engine.connect(), if_exists= 'replace')                
    except Error as e:
            print("Error while import categories to MySQL", e)


In [ ]:
def create_product_df(id):
        data, categories = get_data(id)
        f_data = extract_data_to_feild(data, f_product)
        df = extract_product_data(f_data)
        return df, categories
df, cate = create_product_df(197553025)

In [ ]:
#### Testing Product Mysql
p_id = [101615454,
        114092342,
        114766715,
        120295859,
        123345348]
df,categories = create_product_df(57809866)
print(categories)
for i in p_id:
        df1,categories = create_product_df(i)
        df = df.append(df1,ignore_index=True)
df.drop_duplicates(subset=['id'])
df = df.applymap(str)
import_data_mysql(df, 'Tiki_Product')


***THREAD ADD DATA TO PRODUCT DATA TABLE***
    Create categories dict to store categories id : 100
    multiple thread:
        + get data from categories and dump to database
        + get categories from data get and return to categories

In [ ]:
### create URL categories {id: (name, tick)}, number 100



In [ ]:
### Testing the Functional
def add_status_categories(categories,status):
    def change_status(x):
        x["status"] = status
        return x
    categories = list(map(change_status, categories))
    return pd.DataFrame.from_records(categories)


In [ ]:
def add_categories_data(categories):
	global categories_df
	df1 = add_status_categories(categories,True)
	print('debug1')
	print(categories_df)
	categories_df = categories_df.append(df1,ignore_index=True)
	print('debug2')
	print(categories)



def thread_categories(lock):
	global categories_df
	global categories_list
	if categories_list != []:
		catergories_filter = list(filter(lambda x: False if x['category_id'] in categories_df['category_id'].unique() else False, categories_list))
		print(catergories_filter, categories_list)
		add_categories_data(catergories_filter)
		categories_list = []
	if len(categories_df.index > 100):
		import_categories(categories_df,'Tiki_Product')
	

def thread_add_product(lock):
	global categories_df
	global categories_list
	if len(categories_df.loc[(categories_df['status'] == True)])> 0:
		lock.acquire()
		category = categories_df.loc[(categories_df['status'] == True)].iloc[1]
		if category is not None:
			Name = category['name']
			Id = category['category_id']
			max_page_number = get_max_page_number(Name,Id)
			p_ids = crawling_product_id(Name, Id, max_page_number)
			df,categories = create_product_df(p_ids[0])
			for i in p_ids[1:]:
					df1,categories = create_product_df(i)
					categories_list = categories_list + categories
					print('debug4')
					print(categories_list)
					df = df.append(df1,ignore_index=True)
			df.drop_duplicates(subset=['id'])
			df = df.applymap(str)
			categories_df.loc[categories_df['status'] == True].loc[1,'status'] = False
			print(categories_df)
			import_data_mysql(df, 'Tiki_Product')
			lock.release()
def main_task():
	global categories_df
	global categories_list
	print(categories_df)

	
	# creating a lock
	lock = threading.Lock()

	# creating threads
	t2 = threading.Thread(target=thread_add_product, args=(lock,))
	t1 = threading.Thread(target=thread_categories, args=(lock,))

	# start threads
	t1.start()
	t2.start()

	# wait until threads finish their job
	t1.join()
	t2.join()

if __name__ == "__main__":
	categories = get_product_catagories('https://tiki.vn/dien-thoai-samsung-galaxy-z-flip-4-8gb-128gb-hang-chinh-hang-p194034217.html?itm_campaign=CTP_YPD_TKA_PLA_UNK_ALL_UNK_UNK_UNK_UNK_X.163088_Y.1744616_Z.3253348_CN.Product-Ads-B4Q4&itm_medium=CPC&itm_source=tiki-ads&spid=194034225')
	categories_df = add_status_categories(categories,  True)
	categories_list = []
	while (len(categories_df.index) < 100):
		main_task()
